In [2]:
import os
import osmium
import pyarrow as pa
import pyarrow.parquet as pq
from shapely import from_wkb
from collections import defaultdict
from tqdm.auto import tqdm
import pandas as pd

# ================= CONFIGURATION =================

# TUPLE MAPPING (Key, Value) -> (Class, Group)
# (Key, Value) are taken from tags, all of them are kept at the end.
# (Class, Group) are used to group similar tags together and are defined by us however we want
POI_MAPPING = {
    # # --- ACCOMMODATION ---
    # ("tourism","hotel"): ("hotel","Accommodation"),
    # ("tourism","hostel"): ("hostel","Accommodation"),
    # ("tourism","guest_house"): ("guest_house","Accommodation"),
    # ("tourism","bed_and_breakfast"): ("b_and_b","Accommodation"),
    # ("tourism","apartment"): ("apartment","Accommodation"),
    # ("tourism","chalet"): ("chalet","Accommodation"),
    # ("tourism","camp_site"): ("camp_site","Accommodation"),
    # ("tourism","caravan_site"): ("caravan_site","Accommodation"),
    # ("tourism","alpine_hut"): ("alpine_hut","Accommodation"),

    # --- FOOD ---
    ("amenity","restaurant"): ("restaurant","Food"),
    ("amenity","fast_food"): ("fast_food","Food"),
    ("amenity","cafe"): ("cafe","Food"),
    ("amenity","food_court"): ("food_court","Food"),
    ("amenity","ice_cream"): ("ice_cream","Food"),
    ("amenity","biergarten"): ("biergarten","Food"),

    # --- NIGHTLIFE ---
    ("amenity","bar"): ("bar","Nightlife"),
    ("amenity","pub"): ("pub","Nightlife"),
    ("amenity","nightclub"): ("nightclub","Nightlife"),
    ("amenity","casino"): ("casino","Nightlife"),

    # --- CULTURE ---
    ("tourism","museum"): ("museum","Culture"),
    ("amenity","arts_centre"): ("arts_centre","Culture"),
    ("amenity","theatre"): ("theatre","Culture"),
    ("amenity","cinema"): ("cinema","Culture"),
    ("tourism","gallery"): ("gallery","Culture"),
    ("amenity","library"): ("library","Culture"),

    # --- SIGHTSEEING ---
    ("tourism","attraction"): ("attraction","Sightseeing"),
    ("tourism","viewpoint"): ("viewpoint","Sightseeing"),
    ("tourism","artwork"): ("artwork","Sightseeing"),
    ("historic","monument"): ("monument","Sightseeing"),
    ("historic","memorial"): ("memorial","Sightseeing"),
    ("historic","castle"): ("castle","Sightseeing"),
    ("historic","ruins"): ("ruins","Sightseeing"),
    ("historic","archaeological_site"): ("archaeological","Sightseeing"),
    ("historic","fort"): ("fort","Sightseeing"),
    ("amenity","place_of_worship"): ("place_of_worship","Sightseeing"),

    # --- NATURE ---
    ("natural","beach"): ("beach","Nature"),
    ("natural","peak"): ("peak","Nature"),
    ("natural","volcano"): ("volcano","Nature"),
    ("natural","cave_entrance"): ("cave_entrance","Nature"),
    ("natural","glacier"): ("glacier","Nature"),
    ("leisure","park"): ("park","Nature"),
    ("leisure","garden"): ("garden","Nature"),
    ("leisure","nature_reserve"): ("nature_reserve","Nature"),

    # --- LEISURE / FAMILY ---
    ("leisure","playground"): ("playground","Family"),
    ("tourism","zoo"): ("zoo","Family"),
    ("leisure","water_park"): ("water_park","Family"),
    ("leisure","swimming_pool"): ("swimming_pool","Family"),
    ("leisure","stadium"): ("stadium","Leisure"),
    ("leisure","sports_centre"): ("sports_centre","Leisure"),
    ("leisure","marina"): ("marina","Leisure"),
    ("tourism","picnic_site"): ("picnic_site","Leisure"),

    # --- SUPPLIES ---
    ("shop","supermarket"): ("supermarket","Supplies"),
    ("shop","convenience"): ("convenience","Supplies"),
    ("shop","bakery"): ("bakery","Supplies"),
    ("shop","greengrocer"): ("greengrocer","Supplies"),
    ("shop","general"): ("general_store","Supplies"),

    # --- SHOPPING ---
    ("shop","mall"): ("mall","Shopping"),
    ("shop","department_store"): ("department_store","Shopping"),
    ("shop","clothes"): ("clothes","Shopping"),
    ("shop","gift"): ("gift_shop","Shopping"),
    ("shop","books"): ("bookshop","Shopping"),

    # --- SERVICES ---
    ("tourism","information"): ("tourist_info","Services"),
    ("amenity","toilets"): ("toilets","Services"),
    ("amenity","bank"): ("bank","Services"),
    ("amenity","atm"): ("atm","Services"),
    ("amenity","post_office"): ("post_office","Services"),
    ("shop","laundry"): ("laundry","Services"),
    ("amenity","car_rental"): ("car_rental","Services"),
    ("amenity","bicycle_rental"): ("bicycle_rental","Services"),
    ("amenity","travel_agent"): ("travel_agent","Services"),

    # --- HEALTH & SAFETY ---
    ("amenity","pharmacy"): ("pharmacy","Health"),
    ("amenity","hospital"): ("hospital","Health"),
    ("amenity","clinic"): ("clinic","Health"),
    ("amenity","doctors"): ("doctors","Health"),
    ("amenity","police"): ("police","Health"),

    # --- TRANSPORT ---
    ("railway","station"): ("train_station","Transport"),
    # ("railway","tram_stop"): ("tram_stop","Transport"),
    ("railway","subway_entrance"): ("subway_entrance","Transport"),
    # ("highway","bus_stop"): ("bus_stop","Transport"),
    ("amenity","bus_station"): ("bus_station","Transport"),
    ("amenity","taxi"): ("taxi","Transport"),
    ("amenity","ferry_terminal"): ("ferry_terminal","Transport"),
    ("amenity","airport"): ("airport","Transport"),
}

# Updated Priority Order (Lower number = Higher Priority)
GROUP_PRIORITY = {
    "Accommodation": 1,
    "Sightseeing": 2,
    "Culture": 3,
    "Family": 4,
    "Nightlife": 5,
    "Food": 6,
    "Nature": 7,
    "Transport": 8,
    "Leisure": 9,
    "Shopping": 10,
    "Supplies": 11,
    "Services": 12,
    "Health": 13
}

In [3]:
class LimitReached(Exception):
    pass

class POIHandler(osmium.SimpleHandler):
    def __init__(self, output_path, total_elements=None, batch_size=500_000, max_pois=None, step=1):
        super().__init__()
        self.wkb_factory = osmium.geom.WKBFactory()
        self.columns = defaultdict(list)
        self.output_path = output_path
        self.batch_size = batch_size
        self.writer = None
        self.max_pois = max_pois
        self.step = step

        # Counters
        self.total_pois = 0
        self.batch_count = 0
        self.batch_num = 0
        self.scanned_count = 0

        self.pbar = tqdm(total=total_elements, unit=" elems", desc="Scanning", mininterval=2.0)

        # Schema
        self.schema = pa.schema([
            ("osm_type", pa.string()),
            ("osm_id", pa.int64()),
            ("lat", pa.float64()),
            ("lon", pa.float64()),
            ("poi_class", pa.string()),
            ("poi_group", pa.string()),
            ("tags", pa.map_(pa.string(), pa.string())),
        ])

    def _extract_poi(self, tags, geom_func, osm_type, osm_id):
        # Find all matches
        matches = []
        for k, v in tags:
            if (k, v) in POI_MAPPING:
                p_class, p_group = POI_MAPPING[(k, v)]
                priority = GROUP_PRIORITY.get(p_group, 99)
                matches.append((priority, p_class, p_group))

        if not matches:
            return

        # Picking best match
        # Sort by priority score (lowest number wins)
        matches.sort(key=lambda x: x[0])
        best_match = matches[0]
        poi_class, poi_group = best_match[1], best_match[2]

        # Geometry
        try:
            wkb_data = geom_func()
            geom = from_wkb(wkb_data)
            centroid = geom.centroid
        except Exception:
            return

        # Save all tags
        tag_items = [(t.k, t.v) for t in tags]

        self.columns["osm_type"].append(osm_type)
        self.columns["osm_id"].append(osm_id)
        self.columns["lat"].append(centroid.y)
        self.columns["lon"].append(centroid.x)
        self.columns["poi_class"].append(poi_class)
        self.columns["poi_group"].append(poi_group)
        self.columns["tags"].append(tag_items)

        self.batch_count += 1
        self.total_pois += 1

        if self.total_pois % 1000 == 0:
            self.pbar.set_postfix({"Found POIs": f"{self.total_pois:,}"})

        if self.max_pois and self.total_pois >= self.max_pois:
            raise LimitReached()

        if self.batch_count >= self.batch_size:
            self._flush()

    def _flush(self):
        if not self.columns["osm_id"]: return
        table = pa.Table.from_pydict(self.columns, schema=self.schema)
        if self.writer is None:
            self.writer = pq.ParquetWriter(self.output_path, self.schema, compression='snappy')
        self.writer.write_table(table)
        self.batch_num += 1
        self.columns = defaultdict(list)
        self.batch_count = 0

    def node(self, n):
        self.scanned_count += 1
        if self.scanned_count % 50000 == 0:
            self.pbar.update(50000)

        if self.scanned_count % self.step != 0:
            return

        if not n.tags: return

        try:
            self._extract_poi(n.tags, lambda: self.wkb_factory.create_point(n), "node", n.id)
        except osmium.InvalidLocationError: pass

    def area(self, a):
        self.scanned_count += 1
        if self.scanned_count % 1000 == 0:
             self.pbar.update(1000)

        if self.scanned_count % self.step != 0:
            return

        if not a.tags:
            return

        try:
            self._extract_poi(a.tags, lambda: self.wkb_factory.create_multipolygon(a), "way" if a.from_way() else "relation", a.orig_id())
        except Exception:
            pass

    def close(self):
        self._flush()
        self.pbar.close()
        if self.writer: self.writer.close()

        if os.path.exists(self.output_path):
            size_mb = os.path.getsize(self.output_path) / (1024 * 1024)
            print(f"\nFinished! Extracted {self.total_pois:,} POIs.")
            print(f"Output Size: {size_mb:.2f} MB")

In [4]:
output_file = "south_america_pois.parquet"
input_file = "south-america-251225.osm.pbf"

In [5]:
file_size = os.path.getsize(input_file)
estimated_elements = file_size // 5

# Set step=1, max_pois=None to run the full continent, or change step to consider every step=k node/area
handler = POIHandler(output_file, total_elements=estimated_elements, max_pois=None, step=10)

Scanning:   0%|          | 0/772023810 [00:00<?, ? elems/s]

In [6]:
print(f"Processing {input_file}...")
try:
    handler.apply_file(input_file, locations=True, idx="flex_mem")
except LimitReached:
    print("\nLimit reached.")
finally:
    handler.close()

Processing south-america-251225.osm.pbf...


Scanning:  81%|████████  | 624450000/772023810 [15:22<03:38, 676914.55 elems/s, Found POIs=130,000]


Finished! Extracted 130,206 POIs.
Output Size: 7.93 MB


In [7]:
print("\n--- INSPECTING DATA ---")

size_gb = os.path.getsize(output_file) / (1024**3)
print(f"Parquet Size: {size_gb:.4f} GB")

# Read the file back to verify it works
df = pd.read_parquet(output_file)
print(f"Loaded DataFrame with shape: {df.shape}")
print(df.head(5))

# Show tag structure example
print("\nTag Structure Example:")
print(df.iloc[0]["tags"])


--- INSPECTING DATA ---
Parquet Size: 0.0077 GB
Loaded DataFrame with shape: (130206, 7)
  osm_type    osm_id        lat        lon poi_class poi_group  \
0     node  96183523 -27.339303 -68.877766      peak    Nature   
1     node  96183913 -24.034400 -67.322309      peak    Nature   
2     node  96184413 -27.104828 -68.787111      peak    Nature   
3     node  96184669 -27.459390 -68.980419      peak    Nature   
4     node  96185824 -27.175681 -68.574729   volcano    Nature   

                                                tags  
0                                  [(natural, peak)]  
1  [(ele, 5594), (name, Cerro Rincón), (natural, ...  
2                                  [(natural, peak)]  
3                                  [(natural, peak)]  
4  [(ele, 6501), (name, Volcán Ata), (natural, vo...  

Tag Structure Example:
[('natural', 'peak')]


In [12]:
df.sample(n=80000)

,osm_type,osm_id,lat,lon,poi_class,poi_group,tags
88320,way,545129735,4.441879,-75.189446,park,Nature,"[(leisure, park), (name, Kiosko la Campiña)]"
53530,node,10942773308,-23.215755,-44.713910,restaurant,Food,"[(amenity, restaurant), (check_date, 2025-09-2..."
103911,way,812536218,-22.917554,-42.817948,department_store,Shopping,"[(brand, Americanas), (brand:wikidata, Q306409..."
16793,node,4318833689,-32.593505,-53.380749,place_of_worship,Sightseeing,"[(addr:postcode, 37100), (addr:street, Ismael ..."
47559,node,9237329588,-14.585642,-72.841386,peak,Nature,"[(name, Yuraq Qaqa), (natural, peak), (wikidat..."
...,...,...,...,...,...,...,...
67924,way,171278284,-20.642774,-51.110041,park,Nature,"[(leisure, park), (name, Praça da Matriz)]"
122572,way,1285600759,-5.200072,-35.457677,bank,Services,"[(addr:city, Touros), (addr:housenumber, 46), ..."
48148,node,9364436879,-19.930920,-43.944344,restaurant,Food,"[(addr:housenumber, 607), (amenity, restaurant..."
95114,way,639131078,-7.796446,-34.837811,swimming_pool,Family,"[(leisure, swimming_pool)]"


In [13]:
df[df['osm_type'] == 'relation']

,osm_type,osm_id,lat,lon,poi_class,poi_group,tags
65021,relation,10997410,-29.260844,-71.479075,nature_reserve,Nature,"[(boundary, protected_area), (leisure, nature_..."
65023,relation,6775837,-53.660870,-72.213337,nature_reserve,Nature,"[(boundary, protected_area), (leisure, nature_..."
65028,relation,6342969,-22.979300,-43.212488,sports_centre,Leisure,"[(access, private), (addr:street, Avenida Epit..."
65861,relation,447074,-13.708398,-65.924393,nature_reserve,Nature,"[(addr:country, BO), (boundary, protected_area..."
65868,relation,451551,-3.595277,-79.254555,nature_reserve,Nature,"[(boundary, protected_area), (iba_code, EC068)..."
...,...,...,...,...,...,...,...
130030,relation,5891637,-3.720879,-38.508233,beach,Nature,"[(lifeguard, yes), (name, Praia de Iracema), (..."
130058,relation,6644285,-33.812132,-58.427354,beach,Nature,"[(name, Playa de la Agraciada), (natural, beac..."
130078,relation,19179864,-15.454291,-56.087368,park,Nature,"[(image, https://www.midianews.com.br//storage..."
130117,relation,19972521,-32.948400,-68.856310,park,Nature,"[(leisure, park), (name, Paseo Nuestra Señora ..."


In [14]:
df[df['osm_type'] == 'way']

,osm_type,osm_id,lat,lon,poi_class,poi_group,tags
65016,way,8105832,-23.535091,-46.635314,attraction,Sightseeing,"[(addr:city, São Paulo), (addr:housenumber, 1)..."
65017,way,10241503,-34.562175,-58.458690,park,Nature,"[(addr:city, Ciudad Autónoma de Buenos Aires),..."
65018,way,10429335,-34.576329,-58.503240,park,Nature,"[(leisure, park), (name, Plaza Leandro N. Alem..."
65019,way,10596781,-34.581360,-58.399876,park,Nature,"[(leisure, park), (name, Plaza República de Ch..."
65020,way,11482710,-19.924773,-43.925598,hospital,Health,"[(addr:city, Belo Horizonte), (addr:housenumbe..."
...,...,...,...,...,...,...,...
130201,way,1461562641,-31.152548,-64.475381,swimming_pool,Family,"[(leisure, swimming_pool)]"
130202,way,1461635491,-31.095347,-64.483910,playground,Family,"[(leisure, playground)]"
130203,way,1461644608,-23.493052,-47.511176,pharmacy,Health,"[(addr:housenumber, 1820), (addr:street, Aveni..."
130204,way,1461647495,-41.869160,-73.828990,place_of_worship,Sightseeing,"[(amenity, place_of_worship), (building, resid..."


In [16]:
df[df['osm_type'] == 'node']

,osm_type,osm_id,lat,lon,poi_class,poi_group,tags
0,node,96183523,-27.339303,-68.877766,peak,Nature,"[(natural, peak)]"
1,node,96183913,-24.034400,-67.322309,peak,Nature,"[(ele, 5594), (name, Cerro Rincón), (natural, ..."
2,node,96184413,-27.104828,-68.787111,peak,Nature,"[(natural, peak)]"
3,node,96184669,-27.459390,-68.980419,peak,Nature,"[(natural, peak)]"
4,node,96185824,-27.175681,-68.574729,volcano,Nature,"[(ele, 6501), (name, Volcán Ata), (natural, vo..."
...,...,...,...,...,...,...,...
65011,node,13401933466,-31.311029,-64.462588,doctors,Health,"[(amenity, doctors), (healthcare, doctor), (he..."
65012,node,13405007405,-34.439106,-71.075018,convenience,Supplies,"[(brand, Shell Select), (brand:wikidata, Q1243..."
65013,node,13405274749,-31.804567,-59.166003,toilets,Services,"[(access, yes), (amenity, toilets), (check_dat..."
65014,node,13406341801,-26.993739,-54.483027,clothes,Shopping,"[(addr:housenumber, 80), (addr:postcode, 3354)..."
